In [1]:
#installing necessary libraries
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers, models
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [2]:
#Data Preprocessing
# Define the paths to datasets
train_dir = '/home/micoh/pn_dataset/chest_xray/train'
test_dir = '/home/micoh/pn_dataset/chest_xray/test'

# Image size and batch size
img_size = (150, 150)
batch_size = 32

# Create ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                  zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess training data
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    class_mode='binary')

# Load and preprocess test data
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=img_size,
                                                  batch_size=batch_size,
                                                  class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [3]:
#Building CNN Model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


2023-11-13 19:27:26.274168: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
#Compiling the model
model.compile(optimizer=RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [5]:
epochs = 20

history = model.fit(train_generator,
                    epochs=epochs,
                    validation_data=test_generator)


Epoch 1/30


2023-11-13 19:27:28.566570: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


163/163 [==============================] - ETA: 0s - loss: 0.5277 - accuracy: 0.7805

2023-11-13 19:34:18.778053: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


163/163 [==============================] - 432s 3s/step - loss: 0.5277 - accuracy: 0.7805 - val_loss: 0.7959 - val_accuracy: 0.7147
Epoch 2/30
163/163 [==============================] - 427s 3s/step - loss: 0.2958 - accuracy: 0.8756 - val_loss: 1.0679 - val_accuracy: 0.6362
Epoch 3/30
163/163 [==============================] - 397s 2s/step - loss: 0.2504 - accuracy: 0.8992 - val_loss: 0.4517 - val_accuracy: 0.8237
Epoch 4/30
163/163 [==============================] - 416s 3s/step - loss: 0.2067 - accuracy: 0.9185 - val_loss: 0.4180 - val_accuracy: 0.8542
Epoch 5/30
163/163 [==============================] - 440s 3s/step - loss: 0.1930 - accuracy: 0.9247 - val_loss: 0.3040 - val_accuracy: 0.8894
Epoch 6/30
163/163 [==============================] - 487s 3s/step - loss: 0.1733 - accuracy: 0.9298 - val_loss: 0.3417 - val_accuracy: 0.8942
Epoch 7/30
163/163 [==============================] - 389s 2s/step - loss: 0.1596 - accuracy: 0.9425 - val_loss: 0.4498 - val_accuracy: 0.8510
Epoch 8/30

In [6]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')


2023-11-13 23:00:49.682410: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


20/20 [==============================] - 20s 1s/step - loss: 0.5757 - accuracy: 0.8782
Test accuracy: 0.8782051205635071


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Assuming y_true and y_pred are your true labels and predicted labels
y_true = [1, 0, 1, 1, 0, 1, 0]
y_pred_probs = [0.8, 0.3, 0.7, 0.9, 0.2, 0.6, 0.1]
y_pred = (y_pred_probs > 0.5).astype(int)

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred_probs)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"AUC-ROC: {roc_auc}")

# Make predictions on the test set
y_true = test_generator.classes
y_pred_probs = model.predict(test_generator)
y_pred = (y_pred_probs > 0.5).astype(int)

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

roc_auc = roc_auc_score(y_true, y_pred_probs)

# Print the metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"AUC-ROC: {roc_auc}")

TypeError: '>' not supported between instances of 'list' and 'float'